In [57]:
import pandas as pd
import random

In [58]:
csv = pd.read_csv('input/delitos_2020.csv')

In [59]:
csv.head()

,id_mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_armas,barrio,comuna,latitud,longitud,victimas
0,RH-2020-0000480,2020,Enero,Miércoles,01/01/20,10,Hurto (sin violencia),NaN,NaN,Versalles,10.0,-34.626641,-58.527606,NaN
1,HD-2020-0001119,2020,Enero,Miércoles,01/01/20,22,Homicidio,Femicidio,NaN,Villa Devoto,11.0,-34.611268,-58.525386,1.0
2,RH-2020-0001140,2020,Enero,Miércoles,01/01/20,19,Robo (con violencia),NaN,NaN,Versalles,10.0,-34.625434,-58.525013,NaN
3,RH-2020-0001187,2020,Enero,Miércoles,01/01/20,17,Robo (con violencia),NaN,NaN,Versalles,10.0,-34.625434,-58.525013,NaN
4,RH-2020-0010817,2020,Enero,Miércoles,01/01/20,2,Hurto (sin violencia),NaN,NaN,Villa Devoto,11.0,-34.606789,-58.520726,NaN


In [60]:
# TABLA TIPOS
tipos_csv = csv[['tipo', 'subtipo']]
tipos_csv = tipos_csv.drop_duplicates(ignore_index=True)
tipos_csv = tipos_csv.fillna('Robo')
tipos_csv['id'] = tipos_csv.index + 1

In [61]:
# TABLA TIEMPOS
tiempos_csv = csv[['anio', 'mes', 'dia', 'fecha', 'franja']]
tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('/')[0]))
tiempos_csv = tiempos_csv.drop_duplicates(ignore_index=True)
tiempos_csv['id'] = tiempos_csv.franja + '_' + tiempos_csv.fecha

/var/folders/zk/0yzb9y8s6z71smfld5fjvdjh0000gn/T/ipykernel_10913/2714693099.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('/')[0]))


In [62]:
# TABLA LUGARES
lugares_csv = csv[['barrio', 'comuna', 'latitud', 'longitud']]
# replace SD for NaN
lugares_csv = lugares_csv.replace('SD', "0")
# convert latitud and longitud to numeric
lugares_csv['latitud'] = lugares_csv['latitud'].apply(lambda x: float(x.replace(',', '.')))
lugares_csv['longitud'] = lugares_csv['longitud'].apply(lambda x: float(x.replace(',', '.')))
lugares_csv = lugares_csv.groupby(['barrio', 'comuna']).mean()
lugares_csv = lugares_csv.reset_index()
lugares_csv['id'] = lugares_csv.index + 1

In [63]:
# TABLA TIPO DE ARMA
armas_csv = csv[['uso_armas']]
armas_csv = armas_csv.drop_duplicates(ignore_index=True)
armas_csv['id'] = armas_csv.index + 1

In [64]:
# TABLA DELITOS
delitos_csv = csv

In [65]:
# replace uso_armas for arma_id
delitos_csv = delitos_csv.merge(armas_csv, left_on=['uso_armas'], right_on=['uso_armas'])
delitos_csv = delitos_csv.rename(columns={'id': 'arma_id'})
delitos_csv = delitos_csv.drop(columns=['uso_armas'])

In [66]:
# replace barrio for lugar_id
delitos_csv = delitos_csv.merge(lugares_csv, left_on=['barrio'], right_on=['barrio'])
delitos_csv = delitos_csv.rename(columns={'id': 'lugar_id'})
delitos_csv = delitos_csv.drop(columns=['barrio'])

In [67]:
# replace tipo for tipo_id
delitos_csv = delitos_csv.merge(tipos_csv, left_on=['tipo', 'subtipo'], right_on=['tipo', 'subtipo'])
delitos_csv = delitos_csv.rename(columns={'id': 'tipo_id'})
delitos_csv = delitos_csv.drop(columns=['tipo'])

In [68]:
# replace fecha for tiempo_id
delitos_csv = delitos_csv.merge(tiempos_csv, left_on=['anio', 'mes', 'dia', 'fecha', 'franja'], right_on=['anio', 'mes', 'dia', 'fecha', 'franja'])
delitos_csv = delitos_csv.rename(columns={'id': 'tiempo_id'})
delitos_csv = delitos_csv.drop(columns=['anio', 'mes', 'dia', 'fecha', 'franja'])

In [69]:
# create a victimas column with random values
delitos_csv['victimas'] = delitos_csv.index
delitos_csv['costo'] = delitos_csv.index
delitos_csv['victimas'] = delitos_csv['victimas'].apply(lambda x: random.randint(0, 5))
delitos_csv['costo'] = delitos_csv['costo'].apply(lambda x: random.randint(0, 100000))

In [70]:
delitos_csv = delitos_csv[['tiempo_id', 'lugar_id', 'arma_id', 'tipo_id', 'victimas', 'costo']]
delitos_csv

,tiempo_id,lugar_id,arma_id,tipo_id,victimas,costo
0,21_05/01/20,35,1,6,3,48650
1,21_05/01/20,29,1,4,3,30123
2,21_05/01/20,45,1,7,2,25821
3,20_10/01/20,35,1,6,1,68582
4,20_10/01/20,19,1,6,0,50601
...,...,...,...,...,...,...
13327,4_31/08/20,4,1,9,4,37866
13328,17_27/11/20,4,1,9,1,8100
13329,16_29/08/20,48,1,10,3,44665
13330,18_06/12/20,3,1,11,5,76483


In [71]:
# save csvs to files
tiempos_csv.to_csv('output/tiempos.csv', index=False)
lugares_csv.to_csv('output/lugares.csv', index=False)
armas_csv.to_csv('output/armas.csv', index=False)
delitos_csv.to_csv('output/delitos.csv', index=False)
tipos_csv.to_csv('output/tipos.csv', index=False)